## Baseball Simulator

#### Import modules

In [74]:
import math
from math import sqrt, sin, cos, tan
from scipy.stats import poisson
import matplotlib.pyplot as plt
import numpy as np
import random
%matplotlib inline

#### Define physical properties

In [65]:
# Physical Properties and Conversions
g = 9.81
pi = math.pi

def convert_mph(mph):
    """Converts mph to meters per second."""
    return mph * 0.44704

def convert_meters_ps(mps):
    """Converst meters per second to mph."""
    return mps / 0.44704

def convert_feet(feet):
    """Converts fee to meters."""
    return feet * 0.3048

def convert_meters(meters):
    """Converts meters to feet."""
    return meters / 0.3048

def convert_to_radians(degrees):
    """Converts degrees to radians."""
    return degrees * pi / 180

def convert_to_degrees(radians):
    """Converts radians to degrees."""
    return radians / pi * 180

#### Define Lineup class

In [161]:
class Lineup:
    def __init__(self):
        self.pitcher = None
        self.catcher = None
        self.first = None
        self.second = None
        self.third = None
        self.shortstop = None
        self.right = None
        self.center = None
        self.left = None
        self.dh = None
        
    def set_lineup(self):
        self.lineup = []
        return self.lineup
    
    def set_positions(self):
        pass
    
positioning = {
    'pitcher': (convert_feet(60) / sqrt(2), convert_feet(60) / sqrt(2), convert_feet(6)),
    'catcher': (0, 0, 0),
    'first': (0, 0, 0),
    'second': (0, 0, 0),
    'third': (0, 0, 0),
    'shortstop': (0, 0, 0),
    'right': (0, 0, 0),
    'center': (0, 0, 0),
    'left': (0, 0, 0)
}

#### Define Position Player class

In [449]:
class PositionPlayer:
    def __init__(self, firstname=None, lastname=None, team=None, age=None, salary=None, position=None):
        self.firstname = firstname
        self.lastname = lastname
        self.team = team
        self.age = age
        self.salary = salary
        self.position = position
        self.dominant_hand = "Right"
        self.avg = 0.300
        self.slugging = 0.600
        self.obp = .400
        self.contact = 0.50
        self.plate_discipline = 0.90
        self.running = convert_feet(27)
        self.avg_exit_velocity = 85.0 # in mph
        self.avg_launch_angle = 10 # in degrees
        self.time_to_first = 0
        self.time_to_second = 0
        self.time_to_home = 0
        self.time_to_steal = 0
        self.fielding = 1.000
        self.throwing = convert_mph(95)
        self.fielding_position = None
        self.fieldin_range = pi * self.running
    
    def swing(self, projectile):
        # Select exit velocity
        mu_exit_vel = convert_mph(self.avg_exit_velocity)
        skew = -5
        vels = skewnorm.rvs(a=skew, loc=mu_exit_vel, scale=1, size=10000)
        projectile.exit_velocity = random.choice(vels)
        
        # Select launch angle
        mu_launch = convert_to_radians(self.avg_launch_angle)
        skew = 0
        launch_angles = skewnorm.rvs(a=skew, loc=mu_launch, scale=1, size=10000)
        projectile.launch_angle = random.choice(launch_angles)
        
        # Return exit velo and launch angle
        return projectile.exit_velocity, projectile.launch_angle
        
    def react_to_pitch(self, projectile):
        non_strike_penalty = 0.5
        if projectile.in_strike_zone == False:
            if random.uniform(0, 1) > self.plate_discipline:
                if random.uniform(0, 1) > self.contact:
                    exit_velocity, launch_angle = self.swing(projectile)
                    projectile.destination = projectile.solve_x_distance(exit_velocity, launch_angle, 'metric') * non_strike_penalty
                    print(f'Contact (out of zone, swing). Distance: {projectile.destination:.2f} feet | Exit Velo: {exit_velocity:.2f} mph | Launch angle: {launch_angle:.2f} degrees')
                else:
                    projectile.destination = 0
                    print('Strike (out of zone, swing and miss)')
            else:
                projectile.destination = 0
                print('Ball (out of zone, take)')
        else:
            if random.uniform(0, 1) > self.plate_discipline:
                projectile.destination = 0
                print('Strike (in zone, take)')
            else:
                if random.uniform(0, 1) > self.contact:
                    exit_velocity, launch_angle = self.swing(projectile)
                    projectile.destination = projectile.solve_x_distance(exit_velocity, launch_angle, 'metric')
                    print(f'Contact (in zone, swing). Distance: {projectile.destination:.2f} feet | Exit Velo: {exit_velocity:.2f} mph | Launch angle: {launch_angle:.2f} degrees')
                else:
                    projectile.destination = 0
                    print('Strike (in zone, swing and miss)')
        return projectile
 
    def steal(self, pitcher, catcher):
        pass
    
    def throw(self, target_base):
        pass
    
    def catch(self, ball):
        ball.controlled = True



#### Define Pitcher class

In [432]:
class Pitcher:
    def __init__(self, firstname=None, lastname=None, team=None, age=None, salary=None, position=None):
        self.firstname = firstname
        self.lastname = lastname
        self.team = team
        self.age = age
        self.salary = salary
        self.position = position
        self.dominant_hand = "Right"
        self.velocity_mph = 90
        self.spin_rate = 2400
        self.accuracy = .95
        self.fastball_perc = None
        self.pitches = {
            "fastball": {"frequency": 0.6, "velocity_scaler": 1.00, "spin_scaler": 1.00, "accuracy_scaler": 1.00}, 
            "changup": {"frequency": 0.2, "velocity_scaler": 0.80, "spin_scaler": 0.80, "accuracy_scaler": 0.95}, 
            "slider": {"frequency": 0.1, "velocity_scaler": 0.80, "spin_scaler": 1.20, "accuracy_scaler": 0.80}, 
            "curveball": {"frequency": 0.1, "velocity_scaler": 0.70, "spin_scaler": 1.20, "accuracy_scaler": 0.80}
        }

    def throw_pitch(self):
        pitch = Projectile()
        pitch_pop = list(p1.pitches.keys())
        frq = [pitch_attributes['frequency'] for pitch_type, pitch_attributes in p1.pitches.items()]
        pitch.pitch_type = random.choices(population=pitch_pop,
                                         weights = frq,
                                         k = 1)[0]
        pitch_dic = self.pitches[pitch.pitch_type]
        pitch.controlled = False
        pitch.velocity = convert_mph(self.velocity_mph) * pitch_dic['velocity_scaler']
        pitch.spin = self.spin_rate * pitch_dic['spin_scaler']
        pitch.accuracy = self.accuracy * pitch_dic['accuracy_scaler']
        if random.uniform(0, 1) < pitch.accuracy:
            pitch.in_strike_zone = True
        
        print(f'Pitch type: {pitch.pitch_type}')
        print(f'Spin rate: {pitch.spin:.2f} rpm')
        print(f'Velocity: {pitch.velocity:.1f} mph')
        print(f'Accuracy: {pitch.accuracy:.1%} accurate')
        
        return pitch

#### Define Projectile class

In [447]:
class Projectile:
    def __init__(self):
        """
        All distances in feet
        Velocity in feet per second
        """
        self.controlled = True
        self.spin = None
        self.velocity = None
        self.pitch_type = None
        self.accuracy = None
        self.in_strike_zone = False
        self.height = 0
        self.exit_velocity = 0
        self.destination = 0
        self.launch_angle = 0
        self.spray_angle = 45
        
    def x_displacement(x0, vx, t):
        x = x0 + (vx * t)
        return x

    def y1_func(y0, vy0, vy, t):
        y = y0 + (1/2 * (vy0 + vy) * t)
        return y

    def time_to_peak(vy0, vy):
        t = (vy - vy0) / (-g)
        return t

    def y2_func(y0, vy0, t):
        y = y0 + (vy0 * t) - (1/2 * g * t**2)
        return y

    def vysq_func(vy0, y, y0):
        vysq = vy0**2 - (2 * g * (y - y0))
        return vysq

    def solve_x_distance(self, exit_velocity, launch_angle, measurement='imperial'):
        """
        Inputs: exit velocity in mph and launch angle in degrees.
        Outputs: horizontal distance traveled in the air.
        """
        vel = convert_mph(self.exit_velocity) if measurement=='imperial' else self.exit_velocity
        hgt = convert_feet(self.height) if measurement=='imperial' else self.height
        theta = convert_to_radians(self.launch_angle) if measurement=='imperial' else self.launch_angle
        vx0 = vel * cos(theta) # get x component of exit velocity
        vy0 = vel * sin(theta) # get y component of exit velocity
        t = 2 * time_to_peak(vy0, 0) # calculate total time in the air (at peak, vy = 0)
        x = x_displacement(0, vx0, t) # calculate horizontal displacement based on time in the air
        return x

In [448]:
def simulate_pitch_outcome(batter, pitcher):
    ball = pitcher.throw_pitch()
    ball = batter.react_to_pitch(ball)
    
p1 = Pitcher()
b1 = PositionPlayer()
for _ in range(5):
    simulate_pitch_outcome(batter=b1, pitcher=p1)
    print('\n')

Pitch type: fastball
Spin rate: 2400.00 rpm
Velocity: 40.2 mph
Accuracy: 95.0% accurate
Contact (in zone, swing). Distance: 10.21 feet | Exit Velo: 38.04 mph | Launch angle: 0.03 degrees


Pitch type: fastball
Spin rate: 2400.00 rpm
Velocity: 40.2 mph
Accuracy: 95.0% accurate
Contact (in zone, swing). Distance: -123.45 feet | Exit Velo: 37.64 mph | Launch angle: -0.51 degrees


Pitch type: fastball
Spin rate: 2400.00 rpm
Velocity: 40.2 mph
Accuracy: 95.0% accurate
Strike (in zone, swing and miss)


Pitch type: fastball
Spin rate: 2400.00 rpm
Velocity: 40.2 mph
Accuracy: 95.0% accurate
Contact (in zone, swing). Distance: 56.73 feet | Exit Velo: 37.42 mph | Launch angle: 0.20 degrees


Pitch type: changup
Spin rate: 1920.00 rpm
Velocity: 32.2 mph
Accuracy: 90.2% accurate
Contact (in zone, swing). Distance: 124.13 feet | Exit Velo: 37.68 mph | Launch angle: 1.06 degrees




#### Define Ballpark class

In [ ]:
class Ballpark:
    def __init__(self, name):
        """
        All distances in feet
        """
        self.name = name
        self.homeplate = (convert_feet(0), convert_feet(0), convert_feet(0))
        self.firstbase = (convert_feet(90), convert_feet(0), convert_feet(0))
        self.secondbase = (convert_feet(90), convert_feet(90), convert_feet(0))
        self.thirdbase = (convert_feet(0), convert_feet(90), convert_feet(0))
        self.right_deep = (convert_feet(350), convert_feet(0), convert_feet(10))
        self.center_deep = (convert_feet(350), convert_feet(350), convert_feet(10))
        self.left_deep = (convert_feet(0), convert_feet(350), convert_feet(10)) 

#### Define Game class

In [ ]:
class Game:
    def __init__(self, idx, home_team=None, away_team=None, ballpark=None):
        self.idx = idx
        self.home_team = home_team
        self.away_team = away_team
        self.ballpark = ballpark
        self.runner_on_first = False
        self.runner_on_second = False
        self.runner_on_third = False
        self.home_score = 0
        self.away_score = 0
        self.inning = 0
    
    def simulate_pitch(self):
        pass

#### Testing

In [152]:
p1 = Pitcher()
b1 = PositionPlayer()

print(p1.pitches.items())
print('\n')
for pitch_type, pitch_attributes in p1.pitches.items():
    print(pitch_type, pitch_attributes)
    print(pitch_attributes['frequency'])
print('\n')
frq = [pitch_attributes['frequency'] for pitch_type, pitch_attributes in p1.pitches.items()]
frq

print('\n')
pitch_pop = list(p1.pitches.keys())
pitch_pop

fastball = p1.pitches['fastball']
fastball

fastball['velocity_scaler']

dict_items([('fastball', {'frequency': 0.6, 'velocity_scaler': 1.0, 'spin_scaler': 1.0, 'accuracy_scaler': 1.0}), ('changup', {'frequency': 0.2, 'velocity_scaler': 0.8, 'spin_scaler': 0.8, 'accuracy_scaler': 0.95}), ('slider', {'frequency': 0.1, 'velocity_scaler': 0.8, 'spin_scaler': 1.2, 'accuracy_scaler': 0.8}), ('curveball', {'frequency': 0.1, 'velocity_scaler': 0.7, 'spin_scaler': 1.2, 'accuracy_scaler': 0.8})])


fastball {'frequency': 0.6, 'velocity_scaler': 1.0, 'spin_scaler': 1.0, 'accuracy_scaler': 1.0}
0.6
changup {'frequency': 0.2, 'velocity_scaler': 0.8, 'spin_scaler': 0.8, 'accuracy_scaler': 0.95}
0.2
slider {'frequency': 0.1, 'velocity_scaler': 0.8, 'spin_scaler': 1.2, 'accuracy_scaler': 0.8}
0.1
curveball {'frequency': 0.1, 'velocity_scaler': 0.7, 'spin_scaler': 1.2, 'accuracy_scaler': 0.8}
0.1






1.0